# Introducción
Explorar el formato de los datos del metro

# Config


In [27]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import geopandas as gpd
from datetime import datetime
from google.transit import gtfs_realtime_pb2
import time
from pathlib import Path
import warnings

from realtime.vehicles import load_positions_bus
from config import EXTERNAL_DATA_DIR, PROCESSED_DATA_DIR


In [23]:
#Url Dinamic Data
BUS_VEHICLES_URL = "https://ctb-siri.s3.eu-south-2.amazonaws.com/bizkaibus-vehicle-positions.xml"
NS = {"siri": "http://www.siri.org.uk/siri"}

#Location static data
FOLDER = "Bizkaibus_GPKG"
gpkg_path = EXTERNAL_DATA_DIR / FOLDER / "Bizkaibus_GPKG.gpkg"


# Load data

## Get vehicles

In [5]:
df_metro = load_positions_bus(BUS_VEHICLES_URL, NS)
df_metro

,vehicle_id,lat,lon,timestamp,mode
0,1102,43.271378,-2.941909,2025-12-01 10:51:35+01:00,bus
1,1512,43.271460,-2.931730,2025-12-01 10:51:35+01:00,bus
2,1513,43.363750,-3.016452,2025-12-01 10:51:35+01:00,bus
3,1515,43.264260,-2.934424,2025-12-01 10:51:35+01:00,bus
4,1516,43.262733,-2.947942,2025-12-01 10:51:35+01:00,bus
...,...,...,...,...,...
232,8673,43.396255,-2.698034,2025-12-01 10:51:35+01:00,bus
233,8674,43.334470,-2.676568,2025-12-01 10:51:35+01:00,bus
234,8681,43.313667,-2.675975,2025-12-01 10:51:35+01:00,bus
235,8683,43.317554,-2.675326,2025-12-01 10:51:35+01:00,bus


## Get stations

In [ ]:
stops = gpd.read_file(gpkg_path, layer="Geralekuak_Paradas")

## Get lines

In [25]:
# List all layer names in the GeoPackage
layers = gpd.list_layers(gpkg_path)["name"]

# Exclude the "Geralekuak_Paradas" layer
layers_to_read = [lyr for lyr in layers if lyr != "Geralekuak_Paradas"]

gdfs = []
for lyr in layers_to_read:
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="Non-conformant content for record",
            category=RuntimeWarning,
            module="pyogrio",
        )
        gdf = gpd.read_file(gpkg_path, layer=lyr)

    gdf["layer_name"] = lyr
    gdfs.append(gdf)


# Combine into one GeoDataFrame
lines = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Data management (Code)

## Clean Stations & Stops

,CodigoLinea,Sentido,DenominacionLinea,CodigoTipoRuta,DenominacionTipoRuta,DenominacionRuta,Fecha,geometry,layer_name
0,A0651,I,BILBAO - BALMASEDA,2,Desvío,Sopuerta-Balmaseda (Errefortzu Zerbitzua/Refue...,2022-09-12 08:02:52+00:00,"MULTILINESTRING ((487407.7 4791248.09, 487294....",Ibilbideak_Rutas_NoPrincipales_Ida
1,A0652,I,LANESTOSA - BALMASEDA,6,Secundaria,Lanestosa-Trucios Turtzioz-Artzentales-Zalla-B...,2024-03-21 11:44:30+00:00,"MULTILINESTRING ((464375.53 4785378.66, 464395...",Ibilbideak_Rutas_NoPrincipales_Ida
2,A0653,I,TRUCIOS TURTZIOZ - ARTZENTALES,2,Desvío,Trucios Turtzioz-San Miguel-Santa Cruz-Traslaviña,2015-04-09 07:00:05+00:00,"MULTILINESTRING ((479034.76 4792241.76, 479034...",Ibilbideak_Rutas_NoPrincipales_Ida
3,A0654,I,BALMASEDA - Gurutzeta/Cruces - UPV/EHU,3,Prolongación,Balmaseda-Gurutzeta/Cruces-UPV/EHU,2020-03-30 09:53:58+00:00,"MULTILINESTRING ((483923.1 4782284.5, 483931.2...",Ibilbideak_Rutas_NoPrincipales_Ida
4,A0654,I,BALMASEDA - Gurutzeta/Cruces - UPV/EHU,2,Desvío,Balmaseda-Gurutzeta/Cruces (Institutu),2021-11-30 15:36:25+00:00,"MULTILINESTRING ((483923.1 4782284.5, 483931.2...",Ibilbideak_Rutas_NoPrincipales_Ida
...,...,...,...,...,...,...,...,...,...
404,A3928,V,ARTEA - ZEBERIO - UGAO MIRABALLES,1,Principal,Zeberio-Ugao Miraballes,2016-12-30 23:00:00+00:00,"MULTILINESTRING ((507932.41 4780968.93, 507930...",Ibilbideak_Rutas_Principal_Vuelta
405,A3930,V,BILBAO - GALDAKAO (Autopista),1,Principal,Bilbao-Galdakao (Autopista),2022-07-12 09:53:48+00:00,"MULTILINESTRING ((504128.98 4789805.78, 504134...",Ibilbideak_Rutas_Principal_Vuelta
406,A3931,V,GARAI - DURANGO,1,Principal,Garai-Durango,2019-02-06 11:49:42+00:00,"MULTILINESTRING ((529824.51 4779945.15, 529854...",Ibilbideak_Rutas_Principal_Vuelta
407,A3932,V,GALDAKAO - Metro Bolueta,1,Principal,Galdakao-Metro Bolueta,2022-05-03 13:49:43+00:00,"MULTILINESTRING ((512854.17 4786443.14, 512906...",Ibilbideak_Rutas_Principal_Vuelta


# Plots

In [3]:
# Plots and visualizations

# Save results

In [29]:
#Save results and figures
FOLDER = FOLDER.split("_GPKG")[0]
(PROCESSED_DATA_DIR / FOLDER).mkdir(parents=True, exist_ok=True)
lines.to_file(PROCESSED_DATA_DIR / FOLDER / "bizkaibus_lines.gpkg", layer="lines")
stops.to_file(PROCESSED_DATA_DIR / FOLDER / "bizkaibus_stops.gpkg", layer="stops")